In [ ]:
group_key = ''

In [ ]:
import vk_api 
from vk_api.longpoll import VkLongPoll, VkEventType
from vk_api.utils import get_random_id
from vk_api.keyboard import VkKeyboard, VkKeyboardColor
import pandas as pd
import numpy as np
import logging
import pytest

## Logging

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s,%(levelname)s,%(message)s')

file_handler = logging.FileHandler('billiard.log')
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(formatter)

debug_file_handler = logging.FileHandler('debug_billiard.log', mode = 'w')
debug_file_handler.setLevel(logging.DEBUG)
debug_file_handler.setFormatter(formatter)

logger.addHandler(file_handler)
logger.addHandler(debug_file_handler)

## Dictionary

In [ ]:
CONTEXT_CHANGE_INFO = 'Введи данные, которые хочешь изменить в формате: \n ФИО: Иванов Иван Иванович \n или \n Курс: 1'
CONTEXT_CHOOSE_ACTION = 'Ты можешь проверить свои знания, выбрав конкретную тему, или посмотреть результаты в разделе "Статистика"'
CONTEXT_CHOOSE_TOPIC = 'Выбери тему, по которой хочешь пройти тестирование'
CONTEXT_GREET_USER = 'Привет! С тобой на связи бильярдный клуб МИСИС. Прежде чем приступить к тестам, давай познакомимся. Введи ниже свое ФИО в формате: \n ФИО: Иванов Иван Иванович'
CONTEXT_ENTER_COURSE = 'Теперь введи номер своего курса в формате: \n Курс: 1'
CONTEXT_LAST_QUESTION = 'На этом все! Ты можешь пройти тесты еще раз или посмотреть статистику'
CONTEXT_LETS_START = 'Отлично! Теперь можешь приступать к тестированию'
CONTEXT_LINK = 'Для того, чтобы лучше разобраться в теории бильярда, рекомендуем изучить этот источник: '
CONTEXT_MENU = 'Выберите раздел:'
CONTEXT_OTHER = 'Этот раздел находится на этапе разработки. Мы пришлем вам уведомление, когда здесь появятся новые возможности'
CONTEXT_RIGHT = 'Верно!'
CONTEXT_SUCCESSFUL = 'Из них правильных ответов:'
CONTEXT_TOTAL = 'Всего ответов:'
CONTEXT_USER_NAME = 'фио:'
CONTEXT_COURSE = 'курс:'
CONTEXT_WRONG = 'Неверно. '
TITLE_BACK = 'Назад'
TITLE_BACK_TO_MENU = 'Назад в меню'
TITLE_CHANGE_INFO = 'Редактировать личные данные'
TITLE_CHOOSE_TOPIC = 'Выбрать тему'
TITLE_FOULS = 'Фолы'
TITLE_NEXT = 'Дальше'
TITLE_OTHER = 'Другое'
TITLE_START = 'Начать'
TITLE_STATS = 'Статистика'
TITLE_STOP = 'Завершить'
TITLE_TERMS = 'Терминология бильярда'
TITLE_TESTS = 'Тесты'

## Keyboards

In [ ]:
menu = VkKeyboard()
menu.add_button(TITLE_TESTS, color=VkKeyboardColor.PRIMARY)
menu.add_line()
menu.add_button(TITLE_CHANGE_INFO, color=VkKeyboardColor.PRIMARY)
menu.add_line()
menu.add_button(TITLE_OTHER, color=VkKeyboardColor.SECONDARY)

In [ ]:
test_keyboard = VkKeyboard()
test_keyboard.add_button(TITLE_CHOOSE_TOPIC, color=VkKeyboardColor.PRIMARY)
test_keyboard.add_line()
test_keyboard.add_button(TITLE_STATS, color=VkKeyboardColor.PRIMARY)
test_keyboard.add_line()
test_keyboard.add_button(TITLE_BACK_TO_MENU, color=VkKeyboardColor.SECONDARY)

In [ ]:
theme_keyboard = VkKeyboard()
theme_keyboard.add_button(TITLE_TERMS, color=VkKeyboardColor.PRIMARY)
theme_keyboard.add_line()
theme_keyboard.add_button(TITLE_FOULS, color=VkKeyboardColor.PRIMARY)
theme_keyboard.add_line()
theme_keyboard.add_button(TITLE_BACK, color=VkKeyboardColor.SECONDARY)

In [ ]:
answer_keyboard_4 = VkKeyboard()
answer_keyboard_4.add_button(1, color=VkKeyboardColor.SECONDARY)
answer_keyboard_4.add_button(2, color=VkKeyboardColor.SECONDARY)
answer_keyboard_4.add_button(3, color=VkKeyboardColor.SECONDARY)
answer_keyboard_4.add_button(4, color=VkKeyboardColor.SECONDARY)

In [ ]:
answer_keyboard_2 = VkKeyboard()
answer_keyboard_2.add_button(1, color=VkKeyboardColor.SECONDARY)
answer_keyboard_2.add_button(2, color=VkKeyboardColor.SECONDARY)

In [ ]:
action_choose = VkKeyboard(one_time=True)
action_choose.add_button(TITLE_NEXT, color=VkKeyboardColor.SECONDARY)
action_choose.add_button(TITLE_STOP, color=VkKeyboardColor.SECONDARY)

## Globals

In [ ]:
is_testing = False
right_answer = ''
indx = 0
current_test = pd.DataFrame()
answer_keyboard = VkKeyboard()

In [ ]:
df = pd.read_excel('results.xlsx')
tests_term = pd.read_excel('billiard_tests_term.xlsx')
test_fouls = pd.read_excel('tests_fouls.xlsx')

In [ ]:
link = 'http://www.fbsrf.ru/sites/default/files/02-pravila_igry_v_pul-02-02_.pdf'

## Functions

### Utils

In [ ]:
def get_user_name(text):
    return text[4:].strip()

In [ ]:
def get_course(text):
    return text[5:].strip()

In [ ]:
def is_user_answer(text):
    if text == '1' or text == '2' or text == '3' or text == '4':
        return True
    return False

### Common

In [ ]:
def greeting(user_id, vk):
    vk.messages.send(user_id=user_id,
                     random_id = get_random_id(),
                     message = CONTEXT_GREET_USER)
    return CONTEXT_GREET_USER

In [ ]:
def enter_course(user_id, vk):
    vk.messages.send(user_id=user_id,
                     random_id = get_random_id(),
                     message = CONTEXT_ENTER_COURSE)

In [ ]:
def add_new_user(user_id, text):
    df.loc[len(df.index)] = [user_id,  get_user_name(text), 0, 0, False]

In [ ]:
def first_start(user_id, vk):
    vk.messages.send(user_id=user_id,
                    random_id = get_random_id(),
                    message = CONTEXT_LETS_START,
                    keyboard = menu.get_keyboard())

In [ ]:
def open_menu(user_id, vk):
    vk.messages.send(user_id=user_id,
                    random_id = get_random_id(),
                    message=CONTEXT_MENU,
                    keyboard = menu.get_keyboard())
    return CONTEXT_MENU

In [ ]:
def start(user_id, vk):
    if user_id in df['id'].values:
        return open_menu(user_id=user_id, vk=vk)
    else:
        return greeting(user_id=user_id, vk=vk)

In [ ]:
def send_test_menu(user_id, vk):
    vk.messages.send(user_id=user_id,
                    random_id = get_random_id(),
                    message = CONTEXT_CHOOSE_ACTION,
                    keyboard = test_keyboard.get_keyboard())
    return CONTEXT_CHOOSE_ACTION

In [ ]:
def choose_topic(user_id, vk):
    vk.messages.send(user_id=user_id,
                    random_id = get_random_id(),
                    message = CONTEXT_CHOOSE_TOPIC,
                    keyboard = theme_keyboard.get_keyboard())
    return CONTEXT_CHOOSE_TOPIC

In [ ]:
def change_info(user_id, vk):
    vk.messages.send(user_id=user_id,
                    random_id = get_random_id(),
                    message = CONTEXT_CHANGE_INFO)
    return CONTEXT_CHANGE_INFO

In [ ]:
def other(user_id, vk):
    vk.messages.send(user_id=user_id,
                    random_id = get_random_id(),
                    message = CONTEXT_OTHER)
    return CONTEXT_OTHER

### Test

In [ ]:
def generate_question(vk, user_id):
    global right_answer
    question = current_test['Вопрос'].loc[indx]
    right_answer = str(current_test['Верный ответ'].loc[indx])
    video_id = ''
    if ('video_id' in current_test.columns): 
        video_id = str(current_test['video_id'].loc[indx])
    vk.messages.send(user_id = user_id,
                     random_id = get_random_id(),
                     message = question,
                     keyboard = answer_keyboard.get_keyboard(),
                     attachment = f'video-216098865_{video_id}')
    return question
    
    

In [ ]:
def test_step(user_id, vk):
        global is_testing, indx
        is_testing = True
        generate_question(vk=vk, user_id=user_id)
        indx = indx + 1

In [ ]:
def is_last_question(vk, user_id):
        global indx
        indx = 0
        df.to_excel('results.xlsx', index=False)
        vk.messages.send(user_id = user_id,
                        random_id = get_random_id(),
                        message = CONTEXT_LAST_QUESTION,
                        keyboard = test_keyboard.get_keyboard())

In [ ]:
def answer_is_right(vk, user_id):
    df.loc[df['id'] == user_id, 'right_tries'] += 1
    df.loc[df['id'] == user_id, 'total'] += 1
    vk.messages.send(user_id = user_id,
                     random_id = get_random_id(),
                     message = CONTEXT_RIGHT,
                     keyboard = action_choose.get_keyboard())
    if indx >= len(current_test):
        logger.info(f'end is reached,test,{user_id}')
        is_last_question(vk, user_id)

In [ ]:
def answer_is_wrong(vk, user_id):
    df.loc[df['id'] == user_id, 'total'] += 1
    comment = str(current_test['Объяснение'].loc[indx - 1]) + '\n' + CONTEXT_LINK + link
    vk.messages.send(user_id = user_id,
                     random_id = get_random_id(),
                     message = CONTEXT_WRONG + comment,
                     keyboard = action_choose.get_keyboard())
    if indx >= len(current_test):
        logger.info(f'end is reached,test,{user_id}')
        is_last_question(vk, user_id)

### Statistic

In [ ]:
def send_stats(vk, user_id):
    total = df['total'].loc[df['id'] == user_id].values[0]
    success = df['right_tries'].loc[df['id'] == user_id].values[0]
    message = f'{CONTEXT_TOTAL} {total} \n {CONTEXT_SUCCESSFUL} {success}'
    vk.messages.send(user_id = user_id,
                     random_id = get_random_id(),
                     message = message,
                     keyboard = menu.get_keyboard())

## Main

In [ ]:
vk_session = vk_api.VkApi(token = group_key)
longpoll = VkLongPoll(vk_session)
vk = vk_session.get_api()

In [ ]:

for event in longpoll.listen():
    if event.type == VkEventType.MESSAGE_NEW and event.to_me:
        #chat start
        if event.text == TITLE_START:
            logger.info(f'button start,start,{event.user_id}')
            start(user_id=event.user_id, vk=vk)
        #user name
        if event.text.lower().startswith(CONTEXT_USER_NAME):
            if event.user_id in df['id'].values:
                df['name'].loc[df['id'] == event.user_id] = get_user_name(event.text)
                logger.info(f'user changed name,contact info,{event.user_id}')
                first_start(user_id=event.user_id, vk=vk)
            else:
                add_new_user(user_id=event.user_id, text=event.text)
                logger.info(f'new user,contact info,{event.user_id}')
                enter_course(user_id=event.user_id, vk=vk)
        #course info
        if event.text.lower().startswith(CONTEXT_COURSE):
            try:
                if 1 <= int(get_course(event.text)) < 3:
                    df['PE'].loc[df['id'] == event.user_id] = True
                logger.info(f'PE flag,contact info,{event.user_id}')
                first_start(user_id=event.user_id, vk=vk)
            except:
                logger.error(f'incorrect course number,contact info,{event.user_id}')
        #change info
        if event.text == TITLE_CHANGE_INFO:
            change_info(user_id=event.user_id, vk=vk)
        #menu: TEST
        if event.text == TITLE_TESTS or event.text == TITLE_BACK:
            send_test_menu(user_id=event.user_id, vk=vk)
            logger.info(f'test,menu,{event.user_id}')
        #menu: OTHER
        if event.text == TITLE_OTHER:
            other(user_id=event.user_id, vk=vk)
            logger.info(f'other,menu,{event.user_id}')
        #choose topic
        if event.text == TITLE_CHOOSE_TOPIC:
            choose_topic(user_id=event.user_id, vk=vk)  
            logger.info(f'choose topic,menu,{event.user_id}')
        #test: TERMS
        if event.text == TITLE_TERMS:
            logger.info(f'topic terms,test,{event.user_id}')
            current_test = tests_term.sample(frac=1).reset_index(drop=True)
            answer_keyboard = answer_keyboard_4
            test_step(user_id=event.user_id, vk=vk)
        #test: FOULS
        if event.text == TITLE_FOULS:
            logger.info(f'topic fouls,test,{event.user_id}')
            current_test = test_fouls.sample(frac=1).reset_index(drop=True)
            answer_keyboard = answer_keyboard_2
            test_step(user_id=event.user_id, vk=vk)
        #check answer
        if is_testing:
            if is_user_answer(event.text): 
                if right_answer == event.text:
                    answer_is_right(vk=vk, user_id=event.user_id)
                    is_testing = False
                else:
                    answer_is_wrong(vk=vk, user_id=event.user_id)
                    is_testing = False
            elif event.text != TITLE_FOULS and event.text != TITLE_TERMS:
                logger.warning(f'incorrect answer,test,{event.user_id}')
        #next question
        if event.text == TITLE_NEXT:
            test_step(user_id=event.user_id, vk=vk)
        #stop questions
        if event.text == TITLE_STOP:
            logger.info(f'session interrupted,test,{event.user_id}')
            is_last_question(vk=vk, user_id=event.user_id)
        #get statistics
        if event.text == TITLE_STATS:
            logger.info(f'statistics,statistics,{event.user_id}')
            send_stats(vk=vk, user_id=event.user_id)
        #go back to menu
        if event.text == TITLE_BACK_TO_MENU:
            logger.info(f'back to main,menu,{event.user_id}')
            open_menu(user_id=event.user_id, vk=vk)
            
            